In [ ]:
import requests
import json
import os

In [25]:
def prompt_gemma(prompt):
    url = "http://localhost:11434/v1/chat/completions"
    resp = requests.post(url,
        json={
            "model": "gemma3:1b",
            "stream": False,
            "messages": [
                {"role": "user", "content": f"{prompt}"}
            ]
        })
    return resp.json()
prompt = "When was the last mission on Mars?"
text = prompt_gemma(prompt)
text

{'id': 'chatcmpl-341',
 'object': 'chat.completion',
 'created': 1757915934,
 'model': 'gemma3:1b',
 'system_fingerprint': 'fp_ollama',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "As of today, November 2, 2023, the last official mission to Mars is **Perseverance scouting mission**.\n\nHere's a breakdown:\n\n*   **Perseverance Rover:** The Perseverance rover continued its exploration of Jezero Crater on Mars, focusing on searching for signs of past microbial life and collecting samples for potential future return to Earth. The last data transmission from Perseverance was on **November 19, 2023**.\n\n**Important Note:** While Perseverance continues its work, NASA and other space agencies are actively planning and executing other missions to Mars, so there will be more data and activities coming soon.\n\n**Resources for Updates:**\n\n*   **NASA Mars Exploration Program:** [https://mars.nasa.gov/](https://mars.nasa.gov/)\n*   **Space.com:** [https://www.sp

In [26]:
text_extracted = text['choices'][0]['message']['content']

In [ ]:

def llm_as_a_judge(query, response):
    reference_file = os.path.join(os.getcwd(), "mars.txt")
    with open(reference_file, 'r') as f:
        reference = str(f.readlines())
    text = f"""
    In this task, you will be presented with a query, a reference text and an answer. The answer is
generated to the question based on the reference text. The answer may contain false information. You
must use the reference text to determine if the answer to the question contains false information,
if the answer is a hallucination of facts. Your objective is to determine whether the answer text
contains factual information and is not a hallucination. A 'hallucination' refers to
an answer that is not based on the reference text or assumes information that is not available in
the reference text. Your response should be a single word: either "factual" or "hallucinated", and
it should not include any other text or characters. "hallucinated" indicates that the answer
provides factually inaccurate information to the query based on the reference text. "factual"
indicates that the answer to the question is correct relative to the reference text, and does not
contain made up information. Please read the query and reference text carefully before determining
your response.

    # Query: {query}
    # Reference text: {reference}
    # Answer: {response}
    Is the answer above factual or hallucinated based on the query and reference text?
    """
    url = "http://localhost:11434/v1/chat/completions"
    resp = requests.post(url,
        json={
            "model": "gemma3:1b",
            "stream": False,
            "messages": [
                {"role": "user", "content": f"{text}"}
            ]
        })
    return resp.json()

resp = llm_as_a_judge("What news did NASA confirm about curiosity rover regarding to mars?", "There are aliens on mars!")

resp

['\n', '\n', 'WikipediaThe Free Encyclopedia\n', 'Search Wikipedia\n', 'Search\n', 'Donate\n', 'Create account\n', 'Log in\n', '\n', '\n', 'Wiki Loves Monuments: Photograph a monument, help Wikipedia and win!\n', 'Learn more\n', '\n', 'Contents hide\n', '(Top)\n', 'Natural history\n', '\n', 'Physical characteristics\n', '\n', 'Geography and features\n', '\n', 'Atmosphere\n', '\n', 'Hydrology\n', '\n', 'Orbital motion\n', 'Moons\n', 'Human observations and exploration\n', '\n', 'Habitability and habitation\n', '\n', 'In culture\n', 'See also\n', 'Notes\n', 'References\n', 'Further reading\n', 'External links\n', 'Mars\n', '\n', 'Article\n', 'Talk\n', 'Read\n', 'View source\n', 'View history\n', '\n', 'Tools\n', 'Appearance hide\n', 'Text\n', '\n', 'Small\n', '\n', 'Standard\n', '\n', 'Large\n', 'Width\n', '\n', 'Standard\n', '\n', 'Wide\n', 'Color (beta)\n', '\n', 'Automatic\n', '\n', 'Light\n', '\n', 'Dark\n', 'Featured article\n', 'Page semi-protected\n', 'From Wikipedia, the free enc

{'id': 'chatcmpl-373',
 'object': 'chat.completion',
 'created': 1757916052,
 'model': 'gemma3:1b',
 'system_fingerprint': 'fp_ollama',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Hallucinated.\n\nThe text refers to the idea of aliens on Mars, which is a popular and recurring theme in science fiction and speculation. However, there is no evidence or factual basis supporting the claim that aliens are currently known to inhabit Mars. The text confidently presents a fictional element as a fact.'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 4096,
  'completion_tokens': 62,
  'total_tokens': 4158}}

In [ ]:
from dotenv import load_dotenv

a = load_dotenv()

True

In [48]:
from phoenix.evals import (
    HALLUCINATION_PROMPT_RAILS_MAP,
    HALLUCINATION_PROMPT_TEMPLATE,
    OpenAIModel,
    download_benchmark_dataset,
    llm_classify,
)

model = OpenAIModel(
    model="gpt-5-nano-2025-08-07",
    temperature=0.0,
)

model("what is 2+2?")

RateLimitError: Rate limited: throttling requests to 5.005031385587752 RPS

In [ ]:
#The rails is used to hold the output to specific values based on the template
#It will remove text such as ",,," or "..."
#Will ensure the binary value expected from the template is returned 
rails = list(HALLUCINATION_PROMPT_RAILS_MAP.values())
hallucination_classifications = llm_classify(
    dataframe=df, 
    template=HALLUCINATION_PROMPT_TEMPLATE, 
    model=model, 
    rails=rails,
    provide_explanation=True, #optional to generate explanations for the value produced by the eval LLM